In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import numpy as np
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt

import gcsfs
fs = gcsfs.GCSFileSystem()

import tqdm

# Test Util for Checking/Generating Intermediate Operator Day Analysis Output

In [2]:
# testing parameters: a date that we don't have (saturday)
analysis_date = dt.date(2022, 10, 29)

In [3]:
shared_utils.rt_utils.get_operators(analysis_date,[293])

not yet run: 293


{293: 'not_yet_run'}

In [8]:
date_iso = analysis_date.isoformat()
date_iso

'2022-10-29'

In [13]:
analysis_date = dt.date(2022, 10, 15)

In [14]:
#try a date that we do have
shared_utils.rt_utils.get_operators(analysis_date,[293])

not yet run: 293


{293: 'not_yet_run'}

In [6]:
rt.run_operators(analysis_date,[293])

not yet run: 293
calculating for agency: 293...
getting trips...
complete for agency: 293


In [8]:
rt.run_operators(analysis_date,[293,148])

already ran: 293
not yet run: 148
calculating for agency: 148...
getting trips...
complete for agency: 148
